In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from tqdm.notebook import tqdm
import time

In [2]:
# Le début des relevés est 2013. 

def get_all_data():
    base_url = "https://hubeau.eaufrance.fr/api/v1/vente_achat_phyto/achats/produits"
    params = {
        "type_territoire": "Département",
        "annee_min": 2012,
        "annee_max": 2026,
        "size": 20000,
        "page": 1
    }

    all_data = []

    print(" Accès à l'API en cours... Veuillez patienter.")
    
    start_time = time.time()
    
    while True:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data_page = response.json().get("data", [])
        
        if not data_page:
            break
        
        all_data.extend(data_page)
        params["page"] += 1

        
    end_time = time.time()
    duration = end_time - start_time

    print(f"Nombre de fichiers récupérés :{len(all_data)}")
    return pd.DataFrame(all_data)


In [3]:
df = get_all_data()

 Accès à l'API en cours... Veuillez patienter.
Nombre de fichiers récupérés :909394


In [4]:
df.head()

,achat_etranger,annee,amm,code_territoire,eaj,libelle_territoire,type_territoire,uri_territoire,quantite,unite
0,Non,2023,2130242,78,Non,YVELINES,Département,http://id.eaufrance.fr/dep/78,1014.00,l
1,Non,2023,2110178,24,Non,DORDOGNE,Département,http://id.eaufrance.fr/dep/24,40.00,l
2,Non,2023,9300257,69,Non,RHONE,Département,http://id.eaufrance.fr/dep/69,60.64,l
3,Non,2023,2140252,2B,Oui,HAUTE-CORSE,Département,http://id.eaufrance.fr/dep/2B,122.45,kg
4,Non,2023,2150790,49,Oui,MAINE-ET-LOIRE,Département,http://id.eaufrance.fr/dep/49,866.55,kg


In [23]:
#conservation des colonnes utiles pour augmenter la lisibilité. 
df_propre = df[["amm","annee","code_territoire","eaj","quantite", "unite"]]

In [24]:
df_propre.head()

,amm,annee,code_territoire,eaj,quantite,unite
0,2130242,2023,78,Non,1014.00,l
1,2110178,2023,24,Non,40.00,l
2,9300257,2023,69,Non,60.64,l
3,2140252,2023,2B,Oui,122.45,kg
4,2150790,2023,49,Oui,866.55,kg


In [25]:
# Je vais maintenant supprimer les départements ayant le code territoire 0, 
# représentant une erreur dans la saisie. 
len(df_propre)

909394

In [26]:
(df_propre["code_territoire"] == "00").sum()

np.int64(13182)

In [29]:
df_propre = df_propre[df_propre["code_territoire"] != "00"]

In [30]:
len(df_propre)

896212

In [31]:
# Je vais maintenant convertir les codes en int et changer les noms pour
# les rendre plus compréhensibles
df_propre.info()

<class 'pandas.core.frame.DataFrame'>
Index: 896212 entries, 0 to 909393
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   amm              896212 non-null  object 
 1   annee            896212 non-null  int64  
 2   code_territoire  896212 non-null  object 
 3   eaj              896212 non-null  object 
 4   quantite         896212 non-null  float64
 5   unite            896212 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 47.9+ MB


In [12]:
df_propre = df_propre.rename(columns={
    "code_territoire": "num_département",
    "eaj" : "autorise_jardin",
    "quantite": "quantite_en_kg",
})

df_propre.head()

,amm,annee,num_département,autorise_jardin,quantite_en_kg,unite
0,2130242,2023,78,Non,1014.00,l
1,2110178,2023,24,Non,40.00,l
2,9300257,2023,69,Non,60.64,l
3,2140252,2023,2B,Oui,122.45,kg
4,2150790,2023,49,Oui,866.55,kg


In [13]:
colonnes_entiers = ["amm"] 
for col in colonnes_entiers:
    df_propre[col] = pd.to_numeric(df_propre[col], errors="coerce").astype("Int64")

In [14]:
df_propre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909394 entries, 0 to 909393
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   amm              909394 non-null  Int64  
 1   annee            909394 non-null  int64  
 2   num_département  909394 non-null  object 
 3   autorise_jardin  909394 non-null  object 
 4   quantite_en_kg   909394 non-null  float64
 5   unite            909394 non-null  object 
dtypes: Int64(1), float64(1), int64(1), object(3)
memory usage: 42.5+ MB


Une fois les réglages effectués, on passe au script python qui va enregistrer en base de données. Lancer le script Insertion_data.py

In [15]:
for col in df_propre.select_dtypes(include="object").columns:
    max_len = df_propre[col].astype(str).str.len().max()
    print(f"{col} : VARCHAR({max_len})")


num_département : VARCHAR(3)
autorise_jardin : VARCHAR(3)
unite : VARCHAR(2)


In [16]:
for col in df_propre.select_dtypes(include="int").columns:
    max_val = df_propre[col].max()
    min_val = df_propre[col].min()
    max_digits = max(len(str(abs(max_val))), len(str(abs(min_val))))
    print(f"{col} : INTEGER (min={min_val}, max={max_val}, digits={max_digits})")


amm : INTEGER (min=1030003, max=9900452, digits=7)
annee : INTEGER (min=2013, max=2023, digits=4)


In [17]:
for col in df_propre.select_dtypes(include="float").columns:
    max_val = df_propre[col].max()
    min_val = df_propre[col].min()

    def count_digits(val):
        if pd.isna(val):
            return (0, 0)
        parts = f"{val:.10f}".rstrip("0").split(".")
        return (len(parts[0]), len(parts[1]) if len(parts) > 1 else 0)

    entiers, decimales = zip(*df_propre[col].dropna().map(count_digits))

    print(f"{col} : NUMERIC({max(entiers)+max(decimales)}, {max(decimales)}) "
          f"(min={min_val}, max={max_val})")


quantite_en_kg : NUMERIC(16, 10) (min=0.0, max=878975.0)


In [21]:
print(df_propre["num_département"].value_counts())

num_département
17     13634
51     13546
00     13182
49     13169
59     13041
       ...  
974     1075
971      366
972      363
973      197
976      179
Name: count, Length: 102, dtype: int64
